In [1]:
import numpy as np
from math import gcd
from itertools import permutations
from itertools import combinations
from scipy.optimize import linprog
import matplotlib.pyplot as pp

import pdb
pdb.set_trace()

class word(object):
    """ A word in F_2 
        
        w=b^{beta_n}a^{beta_n}...b^{beta_1}a^{beta_1}
        
        Attributes:
            a_chunks: List of largst contiguous 'a' string
            b_chunks: List of largst contiguous 'b' string

            alpha: array containing exponents of a 
            beta: array containing exponents of b

    """

    def __init__(self, string):
        
        self.string = string
        self.b_chunks = string.split('a')
        self.a_chunks = string.split('b')
        self.h_a = string.count('a')
        self.h_b = string.count('b')
        
    def beta(self):
        exponents = np.array([len(chunk) for chunk in self.b_chunks],dtype='int32') #Ignore zero length chunks
        return np.flip(exponents[exponents>0],0) #Return flipped since word is applied right-to-left
    
    def alpha(self):
        exponents = np.array([len(chunk) for chunk in self.a_chunks],dtype='int32') #Ignore zero length chunks
        return np.flip(exponents[exponents>0],0) #Return flipped since word is applied right-to-left

    def __str__(self):
        return self.string
####################################################################   

def simplify(a,b):
        g = gcd(a,b)
        return (int(a/g),int(b/g))
    
    
def farey(n, extend=1):
    fs = dict()
    for q in range(1,n+1):
        for p in range(1,q):
            
#             fs[float(p)/q] = (c+i,d+i)
            for i in range(extend):
                r=simplify(p+q*i,q)
                fs[float(p+q*i)/q] = r
    return [fs[k] for k in sorted(fs.keys())]
######################################################################
    
def partitions(n):  
    """
    Adapted from https://arxiv.org/abs/0909.2331
    According to the author, this is one of the fastest and efficient algo for printing partition of a number    
    """
    a = [0 for i in range(n + 1)]
    k = 1
    a[1] = n
    while k != 0:
        x = a[k - 1] + 1
        y = a[k] - 1
        k -= 1
        while x <= y:
            a[k] = x
            y -= x
            k += 1
        a[k] = x + y
        yield a[:k + 1]
####################################################################

def fancy_partitions(n,k,min=None,max=None):  
    """
    Produces a list of partition of n into k parts
    each part has value between min and max
    Each partition is a list object of size k that sums to n
    
    Output always shows in ascending order
    """
    
    if min==None:
        min=np.ones(k,dtype=int)
    if max==None:
        max=n*np.ones(k,dtype=int)
    
    print(f'\nWe are partitioning {n} into {k} parts, each of which is at least {min} and at most {max}.'
          +'\nPermutations are ALLOWED as long as each part satisfies the min-max condition.')
    
    if n==sum(max):
        yield max #The fringe case
        return
    
    for partition in partitions(n):
        if (len(partition)==k):
            for solution in set(permutations(partition)):
                if all(min[i]<=solution[i]<=max[i] for i in range(k)):
#                     print(solution)
                    yield solution

    
####################################################################    
            
def setup_lp(w, p, q, c=None, d=None):
    if ((c==None) & (d==None)):
        c = (p*w.h_a + q*w.h_b)
        d = q    
#         print('\nWe are in the fringe Case. Assuming R(w,p/q,1-).\n')

    g=np.dtype(int)
    c,d = simplify(c,d)
    
    n = len(w.alpha())
    
    k = n*d
    
    total = c*q - d*(w.h_a)*p - k 

    if total<=0:
        print("c/d must be at least h_a (p/q) + n . Total must be an integer.")
        return False
    
#     print('\n c is:',c, '\n d is:',d, '\n total is:', total, '\n k is:', k, '\n n is:',n)
    return (int(c) , int(d), int(total), int(k), int(n))


####################################################################

def staircase(w,p,q,c=None,d=None,umin=0):
    
    
# print('alpha exponent array is:',w.alpha(), '\nbeta exponent array is:', w.beta())
    if setup_lp(w, p, q, c, d)==False:
        return ('OOB')
    
    c,d,total,k,n = setup_lp(w, p, q, c, d)
    
    returnvalue=1 #Since u is at most 1

    for l_array in fancy_partitions(total,k,max=[q*(w.beta()[i%n])-1 for i in range(k)]):
        s_array=np.empty(k, dtype='int32')

        for i in range(k):
#             print(sum(np.take(w.alpha(),range(i+1), mode='wrap'))*p + (i+1))
#             print(sum(l_array[:i]))
            s_array[i]=sum(np.take(w.alpha(),range(i+1), mode='wrap'))*p + (i+1) + sum(l_array[:i])

        print('Current partition i.e. l-array is:', l_array)
#         print('s-array is:', s_array)

        A = np.zeros([k, q], dtype=int)

        for i in range(k):
            for j in range(s_array[i]+1,s_array[i]+l_array[i]+1):
                A[i,j%q - 1 ] +=1




        A_lastcol=np.array([[- w.beta()[i%n] for i in range(k)]],dtype=int)

        A = np.concatenate((A, A_lastcol.T), axis=1)

#         print(A)

        A_ub = A
        b_ub = np.zeros(k, dtype=int)

        A_eq = [np.append(np.ones(q, dtype=int),0)]
        b_eq = np.array([1])

        c_lin = np.append(np.zeros(q, dtype=int),1)
        
        bounds = np.row_stack((np.concatenate((np.zeros([q,1]), np.ones([q,1])),axis=1),[umin,1]))

#         res = linprog(c_lin, A_ub, b_ub, A_eq, b_eq, bounds,options={'maxiter':10})

#         print('Optimal value:', res.fun)
#         print('Optimal value:', res.fun, '\nOptimal Solution Vector:', res.x, '\nCurrent Status:', res.status)

#         if res.fun<=returnvalue:
#             returnvalue=res.fun
    
    return (returnvalue)



        
w = word('baaabba')
qMax = 5

# dataX = open(f'dataX_{w}.txt','w+')  

# """Either keep the data points for  future use or plot it"""

# # xvalues=[]
# # yvalues=[]

for p,q in farey(qMax):
#     xvalues.append(p/q)
    fringelength=1-staircase(w,p,q)
#     yvalues.append(fringelength)
    print(f'Fringe length fr_{w}({p}/{q}) is: {fringelength:.10f}')
#     dataX.write("%0.6f , %0.6f \n" % (p/q, fringelength))
            

# pp.figure(figsize=(10,10))

# pp.axis(xmin=0,xmax=1,ymin=0,ymax=1)    
    
# pp.vlines(xvalues,[0], yvalues, linewidth=1e-1)
    
        




--Return--
> <ipython-input-1-84af01298599>(9)<module>()->None
-> pdb.set_trace()
(Pdb) c

We are partitioning 65 into 10 parts, each of which is at least [1 1 1 1 1 1 1 1 1 1] and at most [9, 4, 9, 4, 9, 4, 9, 4, 9, 4].
Permutations are ALLOWED as long as each part satisfies the min-max condition.
Current partition i.e. l-array is: [9, 4, 9, 4, 9, 4, 9, 4, 9, 4]
Fringe length fr_baaabba(1/5) is: 0.0000000000

We are partitioning 10 into 2 parts, each of which is at least [1 1] and at most [7, 3].
Permutations are ALLOWED as long as each part satisfies the min-max condition.
Current partition i.e. l-array is: [7, 3]
Fringe length fr_baaabba(1/4) is: 0.0000000000

We are partitioning 21 into 6 parts, each of which is at least [1 1 1 1 1 1] and at most [5, 2, 5, 2, 5, 2].
Permutations are ALLOWED as long as each part satisfies the min-max condition.
Current partition i.e. l-array is: [5, 2, 5, 2, 5, 2]
Fringe length fr_baaabba(1/3) is: 0.0000000000

We are partitioning 65 into 10 parts, 

In [ ]:
for (c,d) in farey(7,4):
    if ((c/d > 3/7 + 2) and (c/d <= 31/7)):
        print(f'Rinv({c}/{d})=')
        umin=staircase(word('aababbb'),1,7,c,d)
        print(staircase(word('aababbb'),1,7,c,d))

Rinv(5/2)=
0.3333333333333333
Rinv(18/7)=


In [3]:
staircase(word('aababbb'),1,7,5,2)


We are partitioning 25 into 4 parts, each of which is at least [1 1 1 1] and at most [20, 6, 20, 6].
Permutations are ALLOWED as long as each part satisfies the min-max condition.
Current partition i.e. l-array is: (1, 3, 20, 1)
Current partition i.e. l-array is: (20, 1, 1, 3)
Current partition i.e. l-array is: (1, 1, 20, 3)
Current partition i.e. l-array is: (20, 1, 3, 1)
Current partition i.e. l-array is: (20, 3, 1, 1)
Current partition i.e. l-array is: (3, 1, 20, 1)
Current partition i.e. l-array is: (1, 1, 19, 4)
Current partition i.e. l-array is: (1, 4, 19, 1)
Current partition i.e. l-array is: (19, 4, 1, 1)
Current partition i.e. l-array is: (4, 1, 19, 1)
Current partition i.e. l-array is: (19, 1, 1, 4)
Current partition i.e. l-array is: (19, 1, 4, 1)
Current partition i.e. l-array is: (18, 1, 1, 5)
Current partition i.e. l-array is: (1, 5, 18, 1)
Current partition i.e. l-array is: (18, 1, 5, 1)
Current partition i.e. l-array is: (18, 5, 1, 1)
Current partition i.e. l-array is: 

1

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 9., 10.]])

In [ ]:
import pdb
pdb.set_trace()
setup_lp(word('aababbb'),1,7,18,7)
staircase(word('aababbb'),1,7,18,7)

--Return--
> <ipython-input-6-d9c771dd6579>(2)<module>()->None
-> pdb.set_trace()
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2913)run_code()
-> sys.excepthook = old_excepthook
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2929)run_code()
-> outflag = False
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2930)run_code()
-> return outflag
(Pdb) n
--Return--
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2930)run_code()->False
-> return outflag
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2847)run_ast_nodes()
-> for i, node in enumerate(to_run_exec):
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2848)run_ast_nodes()
-> mod = ast.Module([node])
(Pdb) n
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2849)run_ast_nodes()
-> code = c

In [5]:
for part in partitions(5):
    print(part)

[1, 1, 1, 1, 1]
[1, 1, 1, 2]
[1, 1, 3]
[1, 2, 2]
[1, 4]
[2, 3]
[5]


In [4]:
partitions(5)

<generator object partitions at 0x000001456F49BC50>